# Initailize

In [1]:
from init import *
from pyeoskit import eosapi
import evm
from evm import w3

evm.set_chain_id(1)

print('done!')

done!


In [2]:
eosapi.get_balance('helloworld12')

10001000.0

In [3]:
eosapi.get_balance('helloworld11')

10001000.0

# Add eosio.code to active permission

In [4]:
main_account = 'helloworld11'
test_account = 'helloworld12'
evm.set_current_account(test_account)

a = {
    "account": main_account,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":main_account,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {main_account:'owner'})

# Deploy evm smart contract to network

In [5]:
vm_abi = open('./contracts/ethereum_vm/ethereum_vm.abi', 'rb').read()
vm_code = open('./contracts/ethereum_vm/ethereum_vm.wasm', 'rb').read()
r = eosapi.publish_contract('helloworld11', vm_code, vm_abi, vmtype=0, vmversion=0, sign=True, compress=1)
print(r['processed']['elapsed'])
print("Done!")

15255
Done!


# Set a Chain Id

In [6]:
args = {'chainid': 1}
try:
    r = eosapi.push_action(main_account, 'setchainid', args, {main_account:'active'})
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

19719


# # Create an ETH account and bind it to an EOS account

In [7]:
eth_address = None
args = {'account': test_account, 'text': 'hello,world'}
try:
    ret = eosapi.push_action(main_account, 'create', args, {test_account:'active'})
    eth_address = ret['processed']['action_traces'][0]['console']
    print('+++', eth_address)
    print('++++elapsed:', ret['processed']['elapsed'])

except Exception as e:
    if hasattr(e, 'response'):
        parsed = json.loads(e.response)
        print('+++error:\n', json.dumps(parsed, indent=4))
    else:
        print(e)

+++ f375c0d7e20a15b3d35af771763ca166d54cce97
++++elapsed: 518


In [8]:
eth_account = evm.EthAccount('helloworld11', 'f375c0d7e20a15b3d35af771763ca166d54cce97')

In [9]:
eth_account.get_balance()

0.0

In [10]:
eth_account.get_nonce()

1

### Verify eth address method 1

In [11]:
import rlp
import hashlib
from eth_utils import keccak
e = rlp.encode([test_account, 'hello,world'])
h = keccak(e)
if eth_address:
    print(h[12:].hex())
    assert eth_address == h[12:].hex()
else:
    eth_address = h[12:].hex()
    print(eth_address)

f375c0d7e20a15b3d35af771763ca166d54cce97


# Deposit Test

In [12]:
r = eosapi.transfer('helloworld12', 'helloworld11', 10.0, 'deposit')
print('++++console:', r['processed']['action_traces'][0]['inline_traces'][1]['console'])

++++console: +++++eth amount:0


In [13]:
eth_account.get_balance()

10.0

# Withdraw Test

In [14]:
eosapi.get_balance('helloworld12')

10000990.0

### Check balance in ETH account

In [15]:
eth_account.get_balance()

10.0

### Withdraw 1.0000

In [16]:
name = test_account
args = {'account': name, 'amount': '1.0000 SYS'}
try:
    r = eosapi.push_action('helloworld11', 'withdraw', args, {name:'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

++++console: 4,SYS4,SYS
773


### Check balance

In [17]:
eosapi.get_balance(test_account)

10000991.0

### Check balance in binded ETH account

In [18]:
eth_account.get_balance()

9.0

# Deploy an ethereum smart contract

In [19]:
from solcx import compile_source, compile_files

greeter = '''pragma solidity ^0.6.0;
contract Greeter {
    uint value;
    event onSetValue(uint value);
    event onGetValue(uint value);
    
    constructor() public {
        value = 1;
    }

    function getValue() payable public returns (string memory){
        return "hello,world";
    }

    function setValue(uint v) payable public {
        value = v;
        emit onSetValue(v);
    }
}
'''

def compile(contract_source_code, main_class):
    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol[main_class]

    return contract_interface

main_class = '<stdin>:Greeter'
contract_source_code = greeter
contract_interface = compile(contract_source_code, main_class)
bytecode = contract_interface['bin']
abi = contract_interface['abi']
print('Done!')

Done!


In [20]:
import rlp

Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)

args = {'from': w3.toChecksumAddress(eth_address)}

logs = Greeter.constructor().transact(args)
contract_address = logs[0].hex()
contract_address = w3.toChecksumAddress(contract_address)
print(contract_address)


++++elapsed: 1275
0x0633A42c777f64f895dE1B0097de00C8D181A5e9


#### Call setValue in ethereum smart contract

In [21]:
args = {'from': eth_address,'to':contract_address}
logs = Greeter.functions.setValue(0xffaabbcc).transact(args)
print(logs)

++++elapsed: 1064
[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', b'', [b'\x063\xa4,w\x7fd\xf8\x95\xde\x1b\x00\x97\xde\x00\xc8\xd1\x81\xa5\xe9', [b'\xf4\x8a\x1d\xc5~\xef\xa3\xdeD\x06\xe6_\xc0\xfc7{%\xd1\x00\xbd\x06B\x92\x01\xa0\x1a\x9e\x8e\x0e\x1d\x07s'], b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xaa\xbb\xcc']]


# transfer test

In [22]:
eth_account.get_balance()

9.0

In [23]:
eth_contract_account = evm.EthAccount(main_account, contract_address)
eth_contract_account.get_balance()

0.0

### setValue and transfer 1.0 to contract address

In [24]:
print('transfer from ', eth_address, 'to', contract_address)
args = {'from': eth_address,'to':contract_address, 'value':10000}
logs = Greeter.functions.setValue(0xffaabbcc).transact(args)
print(logs)

transfer from  f375c0d7e20a15b3d35af771763ca166d54cce97 to 0x0633A42c777f64f895dE1B0097de00C8D181A5e9
++++elapsed: 1047
[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', b'', [b'\x063\xa4,w\x7fd\xf8\x95\xde\x1b\x00\x97\xde\x00\xc8\xd1\x81\xa5\xe9', [b'\xf4\x8a\x1d\xc5~\xef\xa3\xdeD\x06\xe6_\xc0\xfc7{%\xd1\x00\xbd\x06B\x92\x01\xa0\x1a\x9e\x8e\x0e\x1d\x07s'], b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xaa\xbb\xcc']]


### get address info

In [25]:
eth_account.get_balance()

8.0

In [26]:
eth_contract_account.get_balance()

1.0